In [73]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import root_mean_squared_error
import lightgbm as lgb
import pickle

In [74]:
df_train = pd.read_csv('./data/clean/diamondsdl123fexytdad2xso37.csv',)
df_test = pd.read_csv('./data/clean/testdl123fexydatdso3.7x2.csv',)

y = df_train['price'].values[1:]
X = np.array(df_train.drop(['Unnamed: 0','price'], axis=1))[1:]
X_test = np.array(df_test.drop('Unnamed: 0',axis=1))

In [81]:
df_train.columns

Index(['Unnamed: 0', 'price', 'carat', 'depth', 'table', 'x', 'y', 'z', 'cut',
       'color', 'clarity', 'city', 'x/y', 'td', 'ad', 'cc'],
      dtype='object')

In [76]:
df_test.drop('Unnamed: 0',axis=1)

,carat,depth,table,x,y,z,cut,color,clarity,city,x/y,td,ad
0,0.79,62.7,60.0,5.82,5.89,3.67,3.0,5.0,3.0,1.054812,0.988115,0.956938,0.006279
1,1.20,61.0,57.0,6.81,6.89,4.18,5.0,1.0,5.0,1.040138,0.988389,0.934426,0.006118
2,1.57,62.2,61.0,7.38,7.32,4.57,4.0,3.0,3.0,1.056107,1.008197,0.980707,0.006359
3,0.90,63.8,54.0,6.09,6.13,3.90,3.0,5.0,3.0,1.056107,0.993475,0.846395,0.006182
4,0.50,62.9,58.0,5.05,5.09,3.19,3.0,5.0,5.0,1.054812,0.992141,0.922099,0.006098
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,5.35,5.32,3.30,5.0,6.0,3.0,1.054812,1.005639,0.904685,0.006069
13481,0.71,62.2,55.0,5.71,5.73,3.56,5.0,2.0,4.0,1.048770,0.996510,0.884244,0.006096
13482,0.70,61.6,55.0,5.75,5.71,3.53,5.0,5.0,5.0,1.005179,1.007005,0.892857,0.006040
13483,0.70,58.8,57.0,5.85,5.89,3.45,3.0,5.0,2.0,1.040138,0.993209,0.969388,0.005889


In [77]:
df_train.drop(['Unnamed: 0','price'], axis=1)

,carat,depth,table,x,y,z,cut,color,clarity,city,x/y,td,ad,cc
0,1.21,62.4,58.0,6.83,6.79,4.25,4.0,1.0,4.0,1.069918,1.005891,0.929487,0.006139,4.84
1,0.32,63.0,57.0,4.35,4.38,2.75,3.0,3.0,4.0,1.056107,0.993151,0.904762,0.006107,1.28
2,0.71,65.5,55.0,5.62,5.53,3.65,1.0,4.0,5.0,1.038843,1.016275,0.839695,0.006259,3.55
3,0.41,63.8,56.0,4.68,4.72,3.00,2.0,7.0,3.0,1.056107,0.991525,0.877743,0.006187,1.23
4,1.02,60.5,59.0,6.55,6.51,3.95,5.0,4.0,3.0,1.069918,1.006144,0.975207,0.006056,3.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,1.34,62.7,57.0,7.10,7.04,4.43,5.0,4.0,5.0,1.035822,1.008523,0.909091,0.006052,6.70
40451,2.02,57.1,60.0,8.31,8.25,4.73,2.0,5.0,2.0,1.036038,1.007273,1.050788,0.006229,4.04
40452,1.01,62.7,56.0,6.37,6.42,4.01,5.0,3.0,3.0,1.056107,0.992212,0.893142,0.006159,3.03
40453,0.33,61.9,54.3,4.45,4.47,2.76,5.0,1.0,5.0,1.056107,0.995526,0.877221,0.006011,1.65


In [78]:
df_train.columns

Index(['Unnamed: 0', 'price', 'carat', 'depth', 'table', 'x', 'y', 'z', 'cut',
       'color', 'clarity', 'city', 'x/y', 'td', 'ad', 'cc'],
      dtype='object')

In [79]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=22)

In [89]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
lgb_X = lgb.Dataset(X)

In [90]:
lgb_model = lgb.LGBMRegressor(
    task = 'predict',
    application = 'regression',
    objective = 'root_mean_squared_error',
    boosting_type="gbdt",
    num_iterations = 2500,
    learning_rate = 0.05,
    num_leaves=15,
    tree_learner='feature',
    max_depth =10,
    min_data_in_leaf=7,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='rmse',
    feature_fraction = 0.6,
    random_state=42)

In [80]:
lgb_model.fit(X_train,y_train) 


preds_lgb_model = lgb_model.predict(X_val)
rmse_lgb = root_mean_squared_error(y_val, preds_lgb_model)
print(" RMSE: %f" % (rmse_lgb ))

/home/rjcolgut/.local/lib/python3.10/site-packages/lightgbm/sklearn.py:683: UserWarning: Found 'application' in params. Will use it instead of 'objective' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'objective' argument")
/home/rjcolgut/.local/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the 

/home/rjcolgut/.local/lib/python3.10/site-packages/lightgbm/sklearn.py:683: UserWarning: Found 'application' in params. Will use it instead of 'objective' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'objective' argument")


In [49]:
lgb_model.fit(X,y)

c:\Users\rjcol\.conda\envs\streamlit_env\lib\site-packages\lightgbm\sklearn.py:683: UserWarning: Found 'application' in params. Will use it instead of 'objective' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'objective' argument")
c:\Users\rjcol\.conda\envs\streamlit_env\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the 

LGBMRegressor(application='regression', bagging_fraction=1, bagging_freq=100,
              feature_fraction=0.6, learning_rate=0.05, max_depth=10,
              metric='rmse', min_data_in_leaf=7, num_iterations=2500,
              num_leaves=15, objective='root_mean_squared_error',
              random_state=42, reg_sqrt='True', task='predict',
              tree_learner='feature')

In [60]:
y_pred = lgb_model.predict(X_test)

[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100


c:\Users\rjcol\.conda\envs\streamlit_env\lib\site-packages\lightgbm\sklearn.py:683: UserWarning: Found 'application' in params. Will use it instead of 'objective' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'objective' argument")


In [61]:
output = pd.DataFrame({'id': list(range(len(y_pred))),
                       'price': y_pred.squeeze()})

output.head()

,id,price
0,0,2922.083866
1,1,5561.247998
2,2,9894.319978
3,3,3906.275729
4,4,1663.290318


In [62]:
output.to_csv('./submissions/lgbbasic_dsdl123fexydatdso3x2.csv', index=False)

In [31]:
print('Starting training...')
# train
gbm = lgb.train(
                lgb_train,
                num_boost_round=200,
                valid_sets=lgb_eval,
                callbacks=[lgb.early_stopping(stopping_rounds=15)])

Starting training...


AttributeError: 'LGBMRegressor' object has no attribute 'train'

In [95]:
gridParams_ext = {
    'learning_rate': [0.005, 0.01],
    'n_estimators': [8,16],
    'num_leaves': [6,8], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'objective' : ['regression'],
    'max_bin':[255], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : [500],
    'colsample_bytree' : [0.65],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2],
    'metric': ['rmse'],
    'force_col_wise': [True]}
gridParams_quick = {
    'task' : ['predict'],
    'application' : ['regression'],
    'objective' : ['root_mean_squared_error'],
    'boosting_type' : ["gbdt"],
    'num_iterations' : [2500],
    'learning_rate' : [0.05],
    'num_leaves' : [15],
    'tree_learner' : ['feature'],
    'max_depth' : [10],
    'min_data_in_leaf':[7],
    'bagging_fraction' : [1],
    'bagging_freq' : [100],
    'reg_sqrt' : ['True'],
    'metric' : ['rmse'],
    'feature_fraction' : [0.6],
    'force_col_wise': [True]}
gridParams_art={
        'num_leaves':[70,80,90],
          'min_child_samples':[4,5,6],
          'objective':['regression'],
          'learning_rate':[0.05,0.06, 0.07],
          'boosting_type':['gbdt'],
          'metric':['rmse'],
          'max_depth':[6,7,8],
          #'feature_fraction':[0.8,0.9],
          'force_col_wise' : [True],
          'bagging_fraction': [0.01,0.04],
          'device_type': ['gpu']}
gridParams_xgb= {'colsample_bytree': [0.8,0.9], 'learning_rate': [0.005,0.015], 'max_depth': [6,7,8], 'n_estimators': [800,1600], 'subsample': [0.6,0.9],
    'force_col_wise': [True]}
gridParams_empty = {'objective':['regression']}

In [96]:
mdl = lgb.LGBMRegressor()

In [97]:
#Xs = StandardScaler().fit_transform(X)

In [98]:
grid = GridSearchCV(mdl,gridParams_art, verbose=1, cv=3, n_jobs=-1, scoring='neg_root_mean_squared_error')
# Run the grid
grid.fit(X, y)

best_params = grid.best_params_
best_score = grid.best_score_

print("Best parameters found: ", best_params)
print("Best score found: ", best_score)


Fitting 3 folds for each of 162 candidates, totalling 486 fits
[LightGBM] [Warning] bagging_fraction is set=0.01, subsample=1.0 will be ignored. Current value: bagging_fraction=0.01
[LightGBM] [Warning] bagging_fraction is set=0.01, subsample=1.0 will be ignored. Current value: bagging_fraction=0.01
[LightGBM] [Warning] bagging_fraction is set=0.01, subsample=1.0 will be ignored. Current value: bagging_fraction=0.01
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2238
[LightGBM] [Warning] bagging_fraction is set=0.01, subsample=1.0 will be ignored. Current value: bagging_fraction=0.01
[LightGBM] [Warning] bagging_fraction is set=0.01, subsample=1.0 will be ignored. Current value: bagging_fraction=0.01
[LightGBM] [Warning] bagging_fraction is set=0.01, subsample=1.0 will be ignored. Current value: bagging_fraction=0.01
[LightGBM] [Warning] bagging_fraction is set=0.01, subsample=1.0 will be ignored. Current value: bagging_fraction=0.01
[LightGBM] [Warning] baggi

ValueError: 
All the 486 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
486 fits failed with the following error:
Traceback (most recent call last):
  File "/home/rjcolgut/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/rjcolgut/.local/lib/python3.10/site-packages/lightgbm/sklearn.py", line 1092, in fit
    super().fit(
  File "/home/rjcolgut/.local/lib/python3.10/site-packages/lightgbm/sklearn.py", line 885, in fit
    self._Booster = train(
  File "/home/rjcolgut/.local/lib/python3.10/site-packages/lightgbm/engine.py", line 255, in train
    booster = Booster(params=params, train_set=train_set)
  File "/home/rjcolgut/.local/lib/python3.10/site-packages/lightgbm/b

In [136]:
best_model = grid.best_estimator_

In [ ]:
print('Starting predicting...')
# predict
y_pred = gbm.predict(np.array(X_val))
# eval
rmse_test = root_mean_squared_error(y_val, y_pred) 
print(f'The RMSE of prediction is: {rmse_test}')

Starting predicting...
The RMSE of prediction is: 520.4485025239875


In [151]:
preds = gbm.predict(np.array(X_test))
output = pd.DataFrame({'id': list(range(len(preds))),
                       'price': preds.squeeze()})

In [152]:
output

,id,price
0,0,2866.552767
1,1,5711.512721
2,2,9709.331458
3,3,4027.653518
4,4,1567.500332
...,...,...
13480,13480,1674.217339
13481,13481,2437.809262
13482,13482,3044.305921
13483,13483,2068.585185


In [153]:
output.to_csv('./submissions/lgbdefdl123fexydatdso3x2.csv', index=False)

In [137]:
pickle.dump(best_model, open('./models/lgdl123dlgridtdso3.7x2_511.sav', 'wb'))

In [95]:
train_data = lgb.Dataset('./data/clean/diamondsdlfe2xso.csv', label='price',feature_name=['price', 'carat', 'depth', 'table', 'x', 'y', 'z', 'cut', 'color',
       'clarity', 'city', 'x/y', 'td', 'ad'], categorical_feature= ['cut', 'color','clarity'])

In [86]:
train_data

In [87]:
validation_data = train_data.create_valid('testdl123fexydatdso3x2.csv')

In [88]:
param = {'metric': 'rmse'}
num_round = 10

In [25]:
bst = lgb.train(param, train_data, num_round, valid_sets=[validation_data])

[LightGBM] [Info] Construct bin mappers from text data time 0.02 seconds


TypeError: Wrong type(str) for label.
It should be list, numpy 1-D array or pandas Series